In [2]:
%run use_db_birulevo.ipynb

1. Удаление знаков препинания
2. Удаление хештегов, ссылок
3. Стемминг (mystem)

In [7]:
import re
from pymystem3 import Mystem
mystem = Mystem()

In [8]:
def preprocess_text(txt, punctuation = True, hashtag = True, url = True, stem = True):
    text = txt
    if url:
        text = re.sub('(http\S+)|(www\S+)|(pic\S+)', ' ', text)
    if hashtag:
        text = re.sub('#\w+', ' ', text)
    if punctuation:
        text = re.sub('[^a-zA-Zа-яА-Я1-9@_#]+', ' ', text)
    if stem:
        lemmas = mystem.lemmatize(text)
        text = ''.join(lemmas)
    return (text.strip())

Извлечение из базы данных в текстовый файл сообщений пользователей и меток

In [2]:
def mess_to_txt (user, out_file, punctuation = False, hashtag = False, 
                 url = False, stem = False, errors = False):
    start = time.time()
    query = 'select count(*) from author'
    count = query_with_fetchone(user, query)
    marks = []
    for i in range(1, count[0][0] + 1):
        query = 'select mark from author where id = ' + str(i)
        marks.append(int(query_with_fetchone(user, query)[0][0]))
    try:
        with open(out_file, 'w') as f:
             for i in range(1, count[0][0] + 1):
                    query = 'select mess from messages where author = ' + str(i)
                    list_mess = query_with_fetchone(user, query)
                    for mess in list_mess:
                        st = mess[0]
                        #if errors:
                            # bla-bla, use pyaspeller
                        if (punctuation or hashtag or url or stem):
                            st = preprocess_text(st, punctuation, hashtag, url, stem)
                        else:
                            st = re.sub('\n', ' ', st)
                        if (st == ''):
                            continue
                        f.write(st + ' ')
                    f.write('\n')
    except IOError:
        print("An IOError has occurred!")
    print (time.time() - start)
    return marks

создание модели векторов word2vec из текстового файла

In [10]:
import logging
import multiprocessing
import gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim.models import word2vec
from gensim.models import Word2Vec
import time

In [13]:
def create_model_w2v (file_txt, save_model, size = 200, window = 5, min_count = 3):
    start = time.time()
    data = gensim.models.word2vec.LineSentence(file_txt)
    model = Word2Vec(data, size = size, window = window, min_count = min_count, 
                     workers=multiprocessing.cpu_count())
    end = time.time()
    model.save(save_model)
    return (end - start)

In [14]:
def load_model_w2v(name_model):
    return Word2Vec.load(name_model)

In [15]:
import numpy as np

In [30]:
def create_feature_vec (list_words, model, num_features):
    feature_vec = np.zeros((num_features), dtype = "float32")
    count_words = 0
    set_words_model = set(model.wv.index2word)
    for word in list_words:
        if word in set_words_model:
            count_words += 1
            feature_vec = np.add(feature_vec, model.wv[word])
    feature_vec = np.divide(feature_vec, count_words)
    return (feature_vec)

In [76]:
def text_to_words(text):
    st = re.sub('[^a-zA-Zа-яА-Я1-9@_#]+', ' ', text)
    list_words = st.split()
    for word in list_words:
        word = word.strip()
    return (list_words)

In [1]:
def texts_to_vec (file_txt, model):
    start = time.time()
    num_features = model.wv.vector_size
    text_vec = []
    try:
        with open(file_txt, 'r') as f:
            st = f.readline()
            while (st != ""):
                list_words = text_to_words(st)
                text_vec.append(create_feature_vec(list_words, model, num_features))
                st = f.readline()
    except IOError:
        print("An IOError has occurred!")
    print(time.time() - start)
    return np.array(text_vec, dtype = "float32")

In [1]:
import pandas as pd

In [2]:
def create_data_frame(text_vec, marks):
    df = pd.DataFrame(text_vec)
    df['marks'] = marks
    return (df.loc[(df.marks == -1) | (df.marks == 1)]).reset_index(drop=True)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

In [8]:
def class_random_forest_kf(X, y, n_splitsKF, min_estimators, max_estimators):
    
    kf = KFold(n_splits = n_splitsKF)#, shuffle = True, random_state = 1)
    score_f1 = np.zeros(max_estimators - min_estimators + 1)
    for k in range(min_estimators, max_estimators + 1):
        forest_class = RandomForestClassifier(n_estimators = k)
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.loc[train_index], X.loc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            forest_class.fit(X_train, y_train)
            predict = forest_class.predict(X_test)
            score_f1[k-1] += f1_score(y_test, predict)
    score_f1 /= n_splitsKF
    return score_f1.argmax()

In [25]:
def class_random_forest(df, marks,n_splitsKF, min_estimators, max_estimators):
    start = time.time()
    X = df[new_df.columns[:-1]]
    y = df[marks]
    k = class_random_forest_kf(X, y, n_splitsKF, min_estimators, max_estimators)
    kf = KFold(n_splits = n_splitsKF)#, shuffle = True, random_state = 1)
    score_f1 = 0
    score_acc = 0
    score_rec = 0
    score_prec = 0
    fpr =dict()
    tpr = dict()
    i = 0
    forest_class = RandomForestClassifier(n_estimators = k)
    for train_index, test_index in kf.split(X):
        X_train, X_test = X.loc[train_index], X.loc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        forest_class.fit(X_train, y_train)
        predict = forest_class.predict(X_test)
        score_f1 += f1_score(y_test, predict)
        score_acc += accuracy_score(y_test, predict)
        score_rec += recall_score(y_test, predict)
        score_prec += precision_score(y_test, predict)
        fpr[i], tpr[i], _ = roc_curve(y_test, predict)
        i += 1
        
    score_f1 /= n_splitsKF
    score_acc /= n_splitsKF
    score_rec /= n_splitsKF
    score_prec /= n_splitsKF
    fpr.values
    auc_ = auc(sum(fpr.values())/len(fpr), sum(tpr.values())/len(tpr))
    print("time = ", time.time() - start)
    print("f1 = ", score_f1)
    print("acc = ", score_acc)
    print("rec = ", score_rec)
    print("prec = ", score_prec)
    print("auc = ", auc_)
    plt.plot(sum(fpr.values())/len(fpr), sum(tpr.values())/len(tpr),  color='darkorange', 
             label='ROC curve (area = %0.2f)' %auc_)
    plt.plot([0, 1], [0, 1], color='navy',linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Random Forest Classifier (k = %0.0f)' %k)
    plt.legend(loc="lower right")
    plt.show()
    